Kaggle Hackathon Challenge

In [30]:
import pandas as pd               
import seaborn as sns             
import matplotlib.pyplot as plt   
%matplotlib inline
import re
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords 
import string                     
import numpy as np


In [2]:
# Display.max_colwidth option  set to None to show all text content without truncation
pd.set_option('display.max_colwidth', None)

# Your code to read the DataFrame
train_df = pd.read_csv('train_set.csv')
test_df = pd.read_csv('test_set.csv')


In [3]:
train_df.head(10)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko axhasa ulawulo lwesininzi kunye nokuthath inxaxheba kwabafazi ezi ziquka phakathi kwezinye zazo ikomishoni yokulingana ngokwesini ikomishoni yamalungelo oluntu lomzantsi afrika
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi naphi na kwisebe ngokusekwe kwiimfuno zokusebenza zalo emva kokubonana nomsebenzi kunye okanye imanyano yakhe ukuba ulandulo lomntu onjalo alufanelekanga i-dha mayibize uncedo olufanelekileyo elungelweni layo
2,eng,the province of kwazulu-natal department of transport invites tenders from established contractors experienced in bridge construction for the construction of the kwajolwayo tugela river pedestrian bridge near tugela ferry the duration of the project will be months
3,nso,o netefatša gore o ba file dilo ka moka tše le dumelelanego ka tšona mohlala maleri a magolo a a šomišwago go fihlelela meagong e metelele scaffolds a a bolokegilego lefelo la maleba la go šomela go phela gabotse bjbj
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana u ya nga mulayo wa khomishini ya ndinganyiso ya mbeu u thetshelesa mbilaelo dzine dza tshimbilelana na tshialula u ya nga mbeu nahone i ivhea sa foramu ya thungo u ya nga mulayo wa ndinganyiso
5,nso,dinyakišišo tše tša go dirwa gabedi ka ngwaga di tla lebelela diphetogo tša mebaraka ya mešomo le mekgwa ga mmogo le go thapiwa mešomong ka makaleng ao mešomo ao lego molaong le ao a sego molaong le go hloka me omo
6,tsn,kgetse nngwe le nngwe e e sa faposiwang mo tshekong go ya ka temana ya a e tshwanetse kwa ntle ga fa kgetse e tsholotswe kgotsa e fetiseditswe kwa kgotlatshekelo ya bana ya fetisediwa kwa kgotlatshekelo ya bosiamisi ya ngwana gore a ikarabele mo tatofatsong le go sekiwa ga kgetse go ya ka kgaolo ya
7,ven,mbadelo dze dza laelwa dzi do kwama mahatulele apo a khothe ya madzhisitirata lihoro line la si vhe na mulandu li do vha lone line la wana masheleni ngeno lihoro le la wanwa mulandu li tshi do vha lone line la tea u lifha ayo masheleni o laelwaho nga khothe
8,nso,maloko a dikhuduthamaga a ikarabela mongwe le mongwe go tonakgolo le go thteramelao ya profense malebana le taolo ya dipotfolio tša bona mme maloko a mangwe ka moka a khuduthamaga le wona bjalo a ikrabela ka seboka mabapi le phethagatšo ya mediro ya mmušo wa profense le go merero ya wona
9,tsn,fa le dirisiwa lebone le tshwanetse go bontsha lesedi le le benyabenyang le le bogale jo bo dirang gore le kgone go bonwa ke motho yo o bonang sentle mo leseding le le tlwaelegileng mo sekgaleng sa dimitara di le


In [4]:
train_df.shape

(33000, 2)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [6]:
df_train = train_df.copy() #Let us make a copy of our dataframe to avoid modifying the original dataframe

In [7]:
df_train.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko axhasa ulawulo lwesininzi kunye nokuthath inxaxheba kwabafazi ezi ziquka phakathi kwezinye zazo ikomishoni yokulingana ngokwesini ikomishoni yamalungelo oluntu lomzantsi afrika
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi naphi na kwisebe ngokusekwe kwiimfuno zokusebenza zalo emva kokubonana nomsebenzi kunye okanye imanyano yakhe ukuba ulandulo lomntu onjalo alufanelekanga i-dha mayibize uncedo olufanelekileyo elungelweni layo
2,eng,the province of kwazulu-natal department of transport invites tenders from established contractors experienced in bridge construction for the construction of the kwajolwayo tugela river pedestrian bridge near tugela ferry the duration of the project will be months
3,nso,o netefatša gore o ba file dilo ka moka tše le dumelelanego ka tšona mohlala maleri a magolo a a šomišwago go fihlelela meagong e metelele scaffolds a a bolokegilego lefelo la maleba la go šomela go phela gabotse bjbj
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana u ya nga mulayo wa khomishini ya ndinganyiso ya mbeu u thetshelesa mbilaelo dzine dza tshimbilelana na tshialula u ya nga mbeu nahone i ivhea sa foramu ya thungo u ya nga mulayo wa ndinganyiso


In [8]:


def clean_text_and_remove_outliers(csv_file_path):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)

    # Clean the text in the 'text' column
    df['text'] = df['text'].apply(lambda x: re.sub(r'\W', ' ', str(x)))  # Remove non-word characters
    # Return the cleaned DataFrame
    return df

def vectorize_text(dataframe):
    # Extract the 'text' column from the DataFrame
    texts = dataframe['text']

    # Initialize the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the text data
    text_vectors = vectorizer.fit_transform(texts)

    # Return the text vectors and the vectorizer
    return text_vectors, vectorizer


In [9]:
new_df = clean_text_and_remove_outliers('train_set.csv')
text_vectors = vectorize_text(new_df)

In [10]:
train_df["lang_id"].values

array(['xho', 'xho', 'eng', ..., 'eng', 'xho', 'sot'], dtype=object)

In [11]:
y=df_train['lang_id']
y

0        xho
1        xho
2        eng
3        nso
4        ven
        ... 
32995    tsn
32996    sot
32997    eng
32998    xho
32999    sot
Name: lang_id, Length: 33000, dtype: object

In [12]:
# X_train=df_train['clean_text']
X_train = new_df['text']
X_train

0                                                                                      umgaqo siseko wenza amalungiselelo kumaziko axhasa ulawulo lwesininzi kunye nokuthath inxaxheba kwabafazi ezi ziquka phakathi kwezinye zazo ikomishoni yokulingana ngokwesini ikomishoni yamalungelo oluntu lomzantsi afrika
1                                                      i dha iya kuba nobulumko bokubeka umsebenzi naphi na kwisebe ngokusekwe kwiimfuno zokusebenza zalo emva kokubonana nomsebenzi kunye okanye imanyano yakhe ukuba ulandulo lomntu onjalo alufanelekanga i dha mayibize uncedo olufanelekileyo elungelweni layo
2                                          the province of kwazulu natal department of transport invites tenders from established contractors experienced in bridge construction for the construction of the kwajolwayo tugela river pedestrian bridge near tugela ferry the duration of the project will be months
3                                                                           

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_train,y, test_size=0.2, random_state=42)

In [15]:
len(X_train)

26400

In [16]:
len(X_test)

6600

In [17]:
len(y_train)

26400

## pipeline

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

lang_clf =Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
lang_clf.fit(X_train,y_train)
y_pred = lang_clf.predict(X_test)

print("The accuracy score of the SVC is:", accuracy_score(y_test, y_pred))
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

The accuracy score of the SVC is: 0.9971212121212121


Classification Report:

               precision    recall  f1-score   support

         afr       1.00      1.00      1.00       583
         eng       1.00      1.00      1.00       615
         nbl       0.99      0.99      0.99       583
         nso       1.00      1.00      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       1.00      1.00      1.00       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       1.00      1.00      1.00       609
         zul       0.99      0.99      0.99       590

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [19]:
print(confusion_matrix(y_test,y_pred))

[[582   0   0   0   0   1   0   0   0   0   0]
 [  0 615   0   0   0   0   0   0   0   0   0]
 [  0   0 579   0   0   0   0   0   0   1   3]
 [  0   0   0 623   1   0   1   0   0   0   0]
 [  0   0   0   0 618   0   0   0   0   0   0]
 [  0   0   0   0   0 583   0   0   0   0   1]
 [  1   0   0   0   1   0 596   0   0   0   0]
 [  0   0   0   0   0   0   0 561   0   0   0]
 [  0   0   0   0   0   0   0   0 634   0   0]
 [  0   0   1   0   0   0   0   0   0 607   1]
 [  0   1   4   0   0   0   0   0   0   2 583]]


In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
lr = LogisticRegression(max_iter=1000)

In [22]:
result=lang_clf.predict(X_test)

In [23]:
DAF=pd.DataFrame(result,columns=["lang_id"])
DAF.head()

,lang_id
0,sot
1,nso
2,eng
3,nso
4,nso


In [24]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, result)
print("Accuracy:", accuracy)

Accuracy: 0.9971212121212121


In [25]:
output=pd.DataFrame({"index":test_df["index"]})
submission=output.join(DAF)
submission

,index,lang_id
0,1,sot
1,2,nso
2,3,eng
3,4,nso
4,5,nso
...,...,...
5677,5678,zul
5678,5679,tso
5679,5680,zul
5680,5681,sot


In [26]:
submission.to_csv("submissions.csv",index=False)

In [27]:
# df_newer=betterVect.fit_transform(df_train['lang_id'])
vectorizer = TfidfVectorizer()
df_newer =  vectorizer.fit_transform(df_train['lang_id'])


In [28]:
# from sklearn.preprocessing import StandardScaler

# # Scale the tweet vectors
# scaler = StandardScaler(with_mean=False)
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# X_TEST_scaled= scaler.transform(X_TEST)

In [29]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import RandomForestClassifier
# # Initialize Random Forest classifier
# rf_classifier = RandomForestClassifier(n_estimators=100)

# # Train the classifier
# rf_classifier.fit(X_train_scaled, y_train)

# # Make predictions on the testing data
# y_pred = rf_classifier.predict(X_test_scaled)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Random Forest Accuracy:", accuracy)